In [1]:
!pip install pgeocode

In [2]:
import pgeocode
import pandas as pd
import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
country = pgeocode.Nominatim('in')

In [5]:
country.query_postal_code("500050")

postal_code             500050
country code                IN
place_name         Chandanagar
state_name           Telangana
state_code                  40
county_name          Hyderabad
county_code                536
community_name    Tirumalagiri
community_code             NaN
latitude               17.3898
longitude              78.4699
accuracy                     4
Name: 0, dtype: object

In [17]:
India = country._get_data ('IN')

In [20]:
Postcode = country._index_postal_codes()

In [21]:
Telangana = Postcode[Postcode.state_name == 'Telangana']

In [22]:
Hyderabad = Telangana[Telangana.county_name == 'Hyderabad']

In [23]:
Hyderabad.head()

,country code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
8286,IN,500001,"Gandhi Bhawan (Hyderabad), Seetharampet, Hyder...",Telangana,40,Hyderabad,536.0,Nampally,NaN,17.3862,78.4620,1
8287,IN,500002,"Hyderabad Jubilee H.O, Moghalpura",Telangana,40,Hyderabad,536.0,Charminar,NaN,17.3862,78.4620,1
8288,IN,500003,"Kingsway, Secunderabad H.O",Telangana,40,Hyderabad,536.0,Secunderabad,NaN,17.3862,78.4620,1
8289,IN,500004,"Bazarghat (Hyderabad), Khairatabad H.O, Parish...",Telangana,40,Hyderabad,536.0,Khairatabad,NaN,17.3872,78.4621,4
8290,IN,500005,"Jalapally, Crp Camp (Hyderabad), Keshogiri, Ba...",Telangana,40,Hyderabad,536.0,Hyderabad,NaN,17.3102,78.4997,3


In [25]:
Hyderabad.shape

(104, 12)

In [26]:
Hyderabad.reset_index(inplace= True)

In [28]:
Hyderabad.columns

Index(['index', 'country code', 'postal_code', 'place_name', 'state_name',
       'state_code', 'county_name', 'county_code', 'community_name',
       'community_code', 'latitude', 'longitude', 'accuracy'],
      dtype='object')

In [29]:
Hyderabad = Hyderabad [['postal_code', 'place_name', 'community_name','latitude', 'longitude']]

In [30]:
Hyderabad.head()

,postal_code,place_name,community_name,latitude,longitude
0,500001,"Gandhi Bhawan (Hyderabad), Seetharampet, Hyder...",Nampally,17.3862,78.4620
1,500002,"Hyderabad Jubilee H.O, Moghalpura",Charminar,17.3862,78.4620
2,500003,"Kingsway, Secunderabad H.O",Secunderabad,17.3862,78.4620
3,500004,"Bazarghat (Hyderabad), Khairatabad H.O, Parish...",Khairatabad,17.3872,78.4621
4,500005,"Jalapally, Crp Camp (Hyderabad), Keshogiri, Ba...",Hyderabad,17.3102,78.4997


In [31]:
Hyderabad.community_name.unique()

array(['Nampally', 'Charminar', 'Secunderabad', 'Khairatabad',
       'Hyderabad', 'Asifnagar', 'Golconda', 'Tirumalagiri', 'Amberpet',
       'Rangareddy', 'Balanagar', 'Musheerabad', 'Saidabad',
       'Himayathnagar', 'Rajendranagar', 'Shaikpet', 'Ameerpet',
       'Serilingampally', 'Uppal', 'Hyd', 'Saroornagar', 'Bahadurpura',
       nan, 'Hayathnagar', 'Bandlaguda', 'Qutubullapur', 'Rajendra Nagar',
       'Ghatkesar', 'saroornagar', 'Quthbullapur Mandal', 'Vikarabad'],
      dtype=object)

In [32]:
address = 'Hyderabad, India'

geolocator = Nominatim (user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The Geographical coordinates of Hyderabad are {}, {}.".format(latitude, longitude))

The Geographical coordinates of Hyderabad are 17.38878595, 78.46106473453146.


In [33]:
#mapping Hyderabad using latitude and longitude values
Map_Hyderabad = folium.Map(location=[latitude, longitude], zoom_start =10)

#add markers to map

for lat, lng, place, community in zip(Hyderabad['latitude'], Hyderabad['longitude'], Hyderabad['place_name'], Hyderabad['community_name']):
    label = '{}, {}'.format(community, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity = 0.6,
        parse_html=False).add_to(Map_Hyderabad)
    
Map_Hyderabad    

In [34]:
Hyderabad.groupby("community_name").count()

,postal_code,place_name,latitude,longitude
community_name,,,,
Amberpet,1,1,1,1
Ameerpet,2,2,2,2
Asifnagar,3,3,3,3
Bahadurpura,1,1,1,1
Balanagar,2,2,2,2
Bandlaguda,1,1,1,1
Charminar,5,5,5,5
Ghatkesar,2,2,2,2
Golconda,2,2,2,2


In [35]:
Tirumalagiri = Hyderabad[Hyderabad['community_name']== "Tirumalagiri"].reset_index(drop=True)

In [36]:
Tirumalagiri.head()

,postal_code,place_name,community_name,latitude,longitude
0,500009,Manovikasnagar,Tirumalagiri,17.3862,78.462000
1,500010,"Alwal, Bolarum, Bolarum Bazar",Tirumalagiri,17.3898,78.455267
2,500011,Bowenpally,Tirumalagiri,17.4498,78.555300
3,500014,Hakimpet,Tirumalagiri,17.4010,78.417000
4,500042,Hal (Hyderabad),Tirumalagiri,17.5290,78.683900


In [40]:
Map_Tirumalagiri = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, place, community in zip(Tirumalagiri['latitude'], Tirumalagiri['longitude'], Tirumalagiri['place_name'], Tirumalagiri['community_name']):
    label = '{}, {}'.format(community, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],radius=5, popup=label, color='blue', fill=True, fill_color ='#3186cc', fill_opacity=0.7, parse_html=False).add_to(Map_Tirumalagiri)
    
Map_Tirumalagiri

In [41]:
CLIENT_ID = 'FPHNSFYCTY5Y5IWMNECLQNNEVLQPNCXMP54FYP0J3JRT0RCC' # your Foursquare ID
CLIENT_SECRET = 'YQNXXLU3ZZ5NTBRS1ARJF2RISBJDTBADWUVK4PXH1F2FCH45' # your Foursquare Secret
VERSION = '20191116' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FPHNSFYCTY5Y5IWMNECLQNNEVLQPNCXMP54FYP0J3JRT0RCC
CLIENT_SECRET:YQNXXLU3ZZ5NTBRS1ARJF2RISBJDTBADWUVK4PXH1F2FCH45


In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
Tirumalagiri_Venues = getNearbyVenues(names=Tirumalagiri['place_name'],
                                   latitudes=Tirumalagiri['latitude'],
                                   longitudes=Tirumalagiri['longitude']
                                  )

Manovikasnagar
Alwal, Bolarum, Bolarum Bazar
Bowenpally
Hakimpet
Hal (Hyderabad)
Bahadurpalli, Airforce Academy, Bowrampet, Dindigul, Srirangavaram, Gagillapur
Miyapur, Mathrusri Nagar
Chandanagar
HMT Township
Ramakrishna Puram, Neredmet
BITS, Nisa Hakimpet, Thumkunta, CRPF, Ponnal, Bomraspet, Devaryamjal, Uddemarri
Jntu Kukat Pally, KPHB Colony
Jawahar Nagar, JJ Nagar Colony
Vikasnagar (Hyderabad)
Sainikpuri


In [62]:
Tirumalagiri_Venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Manovikasnagar,17.3862,78.462000,B.R. Timber Panels,17.383007,78.463880,Furniture / Home Store
1,"Alwal, Bolarum, Bolarum Bazar",17.3898,78.455267,Milan Juice Center,17.392266,78.458102,Snack Place
2,"Alwal, Bolarum, Bolarum Bazar",17.3898,78.455267,Apsara Footwear,17.389354,78.453755,Shopping Mall
3,"Alwal, Bolarum, Bolarum Bazar",17.3898,78.455267,Get free mobile recharge everytime you shop vi...,17.391330,78.457978,Business Service
4,"Alwal, Bolarum, Bolarum Bazar",17.3898,78.455267,Summerland,17.392686,78.456209,Juice Bar


In [63]:
#lets figure out how many unique venues category
print('There are {} uniques categories.'.format(len(Tirumalagiri_Venues['Venue Category'].unique())))

There are 21 uniques categories.


In [64]:
#Let us cluster based on neighborhoods
# one hot encoding
Tirumalagiri_Venues_onehot = pd.get_dummies(Tirumalagiri_Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Tirumalagiri_Venues_onehot['Neighborhood'] = Tirumalagiri_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Tirumalagiri_Venues_onehot.columns[-1]] + list(Tirumalagiri_Venues_onehot.columns[:-1])
Tirumalagiri_Venues_onehot = Tirumalagiri_Venues_onehot[fixed_columns]

Tirumalagiri_Venues_grouped = Tirumalagiri_Venues_onehot.groupby('Neighborhood').mean().reset_index()

In [65]:
Tirumalagiri_Venues_grouped.head()

,Neighborhood,ATM,Asian Restaurant,Bar,Bed & Breakfast,Business Service,Café,Chinese Restaurant,Department Store,Diner,...,Furniture / Home Store,Hookah Bar,Indie Movie Theater,Juice Bar,Moving Target,Pizza Place,Platform,Shopping Mall,Snack Place,Train Station
0,"Alwal, Bolarum, Bolarum Bazar",0.0,0.0,0.0,0.00,0.166667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.166667,0.166667,0.0,0.00,0.166667,0.166667,0.00
1,"BITS, Nisa Hakimpet, Thumkunta, CRPF, Ponnal, ...",1.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00
2,"Bahadurpalli, Airforce Academy, Bowrampet, Din...",0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.2,0.2,...,0.0,0.2,0.2,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00
3,Bowenpally,0.0,0.0,1.0,0.00,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00
4,Chandanagar,0.0,0.0,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.25


In [66]:
#Top common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [67]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Tirumalagiri_Venues_sorted = pd.DataFrame(columns=columns)
Tirumalagiri_Venues_sorted['Neighborhood'] = Tirumalagiri_Venues_grouped['Neighborhood']

for ind in np.arange(Tirumalagiri_Venues_grouped.shape[0]):
    Tirumalagiri_Venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tirumalagiri_Venues_grouped.iloc[ind, :], num_top_venues)

Tirumalagiri_Venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alwal, Bolarum, Bolarum Bazar",Shopping Mall,Moving Target,Juice Bar,Business Service,Snack Place,Falafel Restaurant,Train Station,Department Store,Asian Restaurant,Bar
1,"BITS, Nisa Hakimpet, Thumkunta, CRPF, Ponnal, ...",ATM,Falafel Restaurant,Asian Restaurant,Bar,Bed & Breakfast,Business Service,Café,Chinese Restaurant,Department Store,Diner
2,"Bahadurpalli, Airforce Academy, Bowrampet, Din...",Diner,Indie Movie Theater,Hookah Bar,Chinese Restaurant,Department Store,Train Station,Asian Restaurant,Bar,Bed & Breakfast,Business Service
3,Bowenpally,Bar,Train Station,Falafel Restaurant,Asian Restaurant,Bed & Breakfast,Business Service,Café,Chinese Restaurant,Department Store,Diner
4,Chandanagar,Train Station,Platform,Bed & Breakfast,Fried Chicken Joint,Pizza Place,Department Store,Asian Restaurant,Bar,Shopping Mall,Business Service


In [68]:
# set number of clusters
kclusters = 5

Tirumalagiri_Venues_grouped_clustering = Tirumalagiri_Venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tirumalagiri_Venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:24]


array([4, 3, 4, 1, 0, 4, 2], dtype=int32)

In [72]:
# add clustering labels
Tirumalagiri_Venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

Tirumalagiri_merged = Tirumalagiri

Tirumalagiri_merged = Tirumalagiri_merged.join(Tirumalagiri_Venues_sorted.set_index('Neighborhood'), on='place_name')

Tirumalagiri_merged.head() # check the last c

,postal_code,place_name,community_name,latitude,longitude,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,500009,Manovikasnagar,Tirumalagiri,17.3862,78.462000,2.0,2.0,Furniture / Home Store,Train Station,Falafel Restaurant,Asian Restaurant,Bar,Bed & Breakfast,Business Service,Café,Chinese Restaurant,Department Store
1,500010,"Alwal, Bolarum, Bolarum Bazar",Tirumalagiri,17.3898,78.455267,4.0,4.0,Shopping Mall,Moving Target,Juice Bar,Business Service,Snack Place,Falafel Restaurant,Train Station,Department Store,Asian Restaurant,Bar
2,500011,Bowenpally,Tirumalagiri,17.4498,78.555300,1.0,1.0,Bar,Train Station,Falafel Restaurant,Asian Restaurant,Bed & Breakfast,Business Service,Café,Chinese Restaurant,Department Store,Diner
3,500014,Hakimpet,Tirumalagiri,17.4010,78.417000,4.0,4.0,Asian Restaurant,Pizza Place,Café,Department Store,Diner,Train Station,Falafel Restaurant,Bar,Bed & Breakfast,Business Service
4,500042,Hal (Hyderabad),Tirumalagiri,17.5290,78.683900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
Tirumalagiri_merged['Cluster_Labels'].replace(np.NaN, 0, inplace = True)
Tirumalagiri_merged['Cluster_Labels']=Tirumalagiri_merged['Cluster_Labels'].astype('int')

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tirumalagiri_merged['latitude'], Tirumalagiri_merged['longitude'], Tirumalagiri_merged['community_name'], Tirumalagiri_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='rainbow[cluster-1]',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters